<a href="https://colab.research.google.com/github/Witcape/AI_Practice/blob/main/Resnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Add, Activation, ZeroPadding2D, MaxPooling2D, Conv2D, Flatten, BatchNormalization
from tensorflow.keras.activations import relu
from tensorflow.keras.initializers import random_uniform, glorot_uniform

In [3]:
def identity_block(X, f, filters, initializer = random_uniform):
  F1, F2, F3 = filters
  X_Shortcut = X

  X = Conv2D(filters = F1, kernal_size = (1,1), strides = (1,1), padding = 'valid', kernal_initializer = initializer(seed = 0))(X),
  X = BatchNormalization(axis = 3)(X),
  X = Activation('relu')(X,),

  X = Conv2D(filters = F2, kernal_size = (f,f), strides = (1,1), padding = 'same', kernal_initializer = initializer(seed = 0))(X),
  X = BatchNormalization(axis = 3)(X),
  X = Activation('relu')(X),

  X = Conv2D(filters = F3, kernal_size = (1,1), strides = (1,1), padding = 'same', kernal_initializer = initializer(seed = 0))(X),
  X = BatchNormalization(axis = 3)(X),

  X = Add()([X, X_Shortcut]),
  X = Activation('relu')(X),

In [6]:
def convolution_block(X, f, filters, s = 2, initializer = glorot_uniform):
  F1, F2, F3 = filters
  X_Shortcut = X

  X = Conv2D(filters = F1, kernel_size = (1,1), strides = (s,s), padding = 'valid', kernel_initializer = initializer(seed = 0))(X),
  X = BatchNormalization(axis = 3)(X),
  X = Activation('relu')(X,),

  X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), padding = 'same', kernel_initializer = initializer(seed = 0))(X),
  X = BatchNormalization(axis = 3)(X),
  X = Activation('relu')(X),

  X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), padding = 'same', kernel_initializer = initializer(seed = 0))(X),
  X = BatchNormalization(axis = 3)(X),

  X = Add()([X_shortcut,X])
  X = Activation('relu')(X)

  X = Add()([X, X_Shortcut]),
  X = Activation('relu')(X),